In [3]:


import pandas as pd
import random
import pprint

from transformers import AutoModel, AutoTokenizer

from torch import nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score

ModuleNotFoundError: No module named 'pandas'

In [ ]:
for i in range(100)

In [164]:
data = pd.read_excel("data.xlsx", engine="openpyxl")#.iloc[:100]
# data["label_idx"] = [random.choice([0,1,2,3,4]) for _ in range(100)]
display(data.head(2))
data.shape

,sentence,label_idx
0,후기 남길게요 오븐 스파게티 서비스로 주세요,0
1,혹시 코스요리에서 파스타만 포장 가능할까요?,0


(15726, 2)

In [112]:
all_indices = data.index

In [113]:
train_size = int(0.8 * len(all_indices))
val_size = len(all_indices) - train_size
train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8 * len(all_indices)):]

num_classes = max(data["label_idx"].tolist())+1

In [114]:
train_idx, val_idx

(RangeIndex(start=0, stop=80, step=1), RangeIndex(start=80, stop=100, step=1))

In [115]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

In [116]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터셋 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.encodings = [
            tokenizer.encode_plus(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512) for sentence in df["sentence"].tolist()
        ]
        self.labels = df["label_idx"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.encodings[idx], self.labels[idx])
        # return to_return

In [117]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [118]:
train_dataset = TextClassificationDataset(train_data, tokenizer)
val_dataset = TextClassificationDataset(val_data, tokenizer)

In [119]:
train_dataset[0]

({'input_ids': tensor([[    0,  7541, 20206,  7187, 18301, 23115,  3838,  2200, 27616,  2182,
              2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 3)

In [131]:
batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [132]:
model = AutoModel.from_pretrained("klue/roberta-small")

class CustomModel(nn.Module):
    def __init__(self, bert_model, num_classes, hidden_size):
        super(CustomModel, self).__init__()
        self.model = bert_model
        self.num_classes = num_classes
        
        self.fc = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, _input):
        output = self.model(**_input)["pooler_output"]
        output = self.fc(output[0])
        return output

custom_model = CustomModel(model, num_classes, 768)
optimizer = AdamW(custom_model.parameters(), lr=0.001)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dwook.kim/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [162]:
loss_f = nn.CrossEntropyLoss()
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
    # 모델을 학습 모드로 설정합니다.
    model.train()
    for i, (_input, _label) in enumerate(train_loader):
        _input = {k:v.squeeze(0) for k, v in _input.items()}
        _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
        
        outputs = custom_model(_input)
        loss = loss_f(F.softmax(outputs), _label)
        loss.backward() # 역전파를 수행합니다.
        optimizer.step()
        optimizer.zero_grad()
    
    print(outputs)
    model.eval()
    all_predictions = []
    all_labels = []

    for i, (_input, _label) in enumerate(val_loader):
        _input = {k:v.squeeze(0) for k, v in _input.items()}
        _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
        
        outputs = custom_model(_input)
        #loss = loss_f(F.softmax(outputs), _label)
        all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
        all_labels.append( torch.argmax(F.softmax(_label)).item() )
        


    print(accuracy_score(all_labels, all_predictions))
    print(f1_score(all_labels, all_predictions, average="weighted"))
    

Epoch 1/3
----------


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = loss_f(F.softmax(outputs), _label)


tensor([-3.4785, -3.5743, 11.4758, -2.6121, -2.7977], grad_fn=<ViewBackward0>)


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_labels.append( torch.argmax(F.softmax(_label)).item() )


0.1
0.01818181818181818
Epoch 2/3
----------


...

0.1
0.01818181818181818
Epoch 3/3
----------


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = loss_f(F.softmax(outputs), _label)


tensor([-3.3328, -3.3854, 10.9299, -2.3194, -2.6250], grad_fn=<ViewBackward0>)


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/4052897966.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_labels.append( torch.argmax(F.softmax(_label)).item() )


0.1
0.01818181818181818


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_labels.append( torch.argmax(F.softmax(_label)).item() )


In [135]:
# 학습하기 이전 inference 결과
outputs

tensor([-0.1002,  0.0965,  0.0802, -0.0014, -0.3126], grad_fn=<ViewBackward0>)

In [138]:
# 학습하기 이전 inference 결과
outputs

tensor([-3.8254, -3.8895, 12.4735, -2.9632, -3.1166], grad_fn=<ViewBackward0>)

In [145]:
_label

tensor([0., 0., 0., 0., 1.])

In [144]:
F.softmax(outputs)

/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3142704713.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(outputs)


tensor([8.3459e-08, 7.8275e-08, 1.0000e+00, 1.9766e-07, 1.6954e-07],
       grad_fn=<SoftmaxBackward0>)

In [143]:
torch.argmax(F.softmax(outputs)).item()

/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3085399197.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.argmax(F.softmax(outputs)).item()


2

tensor([-3.6564, -3.7143, 11.9799, -2.7231, -2.9375], grad_fn=<ViewBackward0>)